In [1]:
import pandas as pd
from helpers import *
import os
from collections import Counter

In [2]:
cong_zip_df = pd.read_csv("data/zccd_hud.csv")

In [3]:
pp_df = pd.read_excel("data/propublica_api_members.xlsx")

In [4]:
zq = pd.read_excel('data/zip_queries.xlsx')

In [5]:
cong_zip_df['cd'] = cong_zip_df.cd.apply(lambda x: 'At Large' if x == 0 else str(x))
cong_zip_df['zip'] = cong_zip_df.zip.apply(lambda z: ((5 - len(str(z))) * '0') + str(z))
cong_zip_df.head(2)

,state_fips,state_abbr,zip,cd
0,1,AL,30165,3
1,1,AL,31905,3


In [6]:
zip_queries = pd.read_excel('data/zip_queries.xlsx')
zip_queries['zip'] = zip_queries.zip.apply(lambda z: ((5 - len(str(z))) * '0') + str(z))

In [7]:
d = cong_zip_df.merge(pp_df, left_on = ['state_abbr', 'cd'], right_on = ['state', 'district'])

In [8]:
d.to_csv("full_zip_cong.csv")

Match as many of the 2000 zipcodes with the apprmopriate district and representatives. Could only find match with 1916. Next time we do data collection, let's just start with things that match. 

In [9]:
file_names = os.listdir("results/3_9_zipcode")

In [30]:
d = d[d['zip'].isin(zip_queries.zip.values)]

In [11]:
def getLastNames(df, z):
    return df.loc[df['zip'] == z, 'last_name'].values

In [12]:
div_df = createZipcodeDivDf(d, '3_9_zipcode')

20 40 60 80 100 120 140 160 180 200 220 240 260 280 300 320 340 360 380 400 420 440 460 480 500 520 540 560 580 600 620 640 660 680 700 720 740 760 780 800 820 840 860 880 900 920 940 960 980 1000 1020 1040 1060 1080 1100 1120 1140 1160 1180 1200 1220 1240 1260 1280 1300 1320 1340 1360 1380 1400 1420 1440 1460 1480 1500 1520 1540 1560 1580 1600 1620 1640 1660 1680 1700 1720 1740 1760 1780 1800 1820 1840 1860 1880 1900 1920 1940 1960 1980 2000 2020 2040 2060 2080 2100 2120 2140 2160 2180 2200 2220 2240 2260 2280 2300 2320 2340 2360 2380 2400 2420 2440 

In [21]:
div_df.shape

(1916, 4)

In [13]:
div_text_df = createTextDf(div_df)

In [14]:
def createBoolNameDfZip(text_df):
    '''return Dataframe that returns True where the appropriate congressperson's
    name is in the div text'''
    name_dict = {}
    for i, row in text_df.iterrows():
           name_dict[i] = [isNameInResultZip(result_text,i) for result_text in row]
    return pd.DataFrame.from_dict(name_dict, orient='index')

In [15]:
def isNameInResultZip(div_text, z):
    '''test if the congressperson's last name appears in the text of the result div
    if this is an analysis based on congressional district expect a string for cong_id,
    if this is an analysis based on zipcode, expect a list for cong_id'''
    last_names = d.loc[d.zip == z, 'last_name']
    return any([name in div_text for name in set(last_names.values)])

In [16]:
createBoolNameDfZip(div_text_df).to_csv('zip_name_df_3_9.csv')

In [33]:
rep_names = d.last_name.values
def hasRepName(div_text): 
    names = set()
    for word in div_text.split():
        if word in rep_names:
            names.add(word)
    return names

In [34]:
rep_names = div_text_df.applymap(hasRepName)

In [35]:
rep_names.head(2)

,congress,congressperson,rep,representative
31905,{},{Bishop},{},{}
35044,{},{Sewell},{},{}


In [17]:
def isNameInSet(s,cong_id):
    '''test if the congressperson's last name appears in the list of names'''
    last_names = d.loc[d.zip == z, 'last_name']
    for name in s: 
        if name in last_names:
            return True
    return False

In [18]:
t = {} 
for z in d.zip.values: 
    t[z] = d.loc[d.zip == z, 'last_name'].values

l_df = pd.DataFrame(data=[t.values()], columns = t.keys()).T
l_df = l_df.reset_index()

In [19]:
def otherNamesDf(rep_names_df):
    name_dict = {}
    for i, row in rep_names_df.iterrows():
        name_dict[i] = []
        for s in row: 
            
            if isNameInResultZip(s, i):
                name_dict[i].append(set())
            else: 
                name_dict[i].append(s)
            
    df = pd.DataFrame.from_dict(name_dict, orient='index')
    df = df.reset_index()
    return df

In [36]:
other_names = otherNamesDf(rep_names)

In [37]:
other_names.to_csv('3_9_other_rep_names.csv')

In [25]:
l_df.head(2)

,index,0
0,31905,"[Rogers, Bishop]"
1,35044,[Rogers]


In [26]:
other_names.merge(l_df, on='index').to_excel('other_reps_zipcode_3_9.xlsx')

In [29]:
Counter([j  for el in other_names.merge(l_df, on='index')[['0_x', 1, 2, 3]].values for l in el for j in l if l != set()]).most_common()

[('Cook', 54),
 ('Scott', 30),
 ('Sherman', 21),
 ('Lee', 21),
 ('Johnson', 18),
 ('Williams', 15),
 ('Smith', 15),
 ('Harris', 14),
 ('Kim', 13),
 ('McNerney', 13),
 ('Green', 12),
 ('Hastings', 12),
 ('Evans', 12),
 ('Lowenthal', 11),
 ('Hill', 11),
 ('Serrano', 11),
 ('Cleaver', 10),
 ('Thompson', 9),
 ('Costa', 9),
 ('Kelly', 9),
 ('Ryan', 9),
 ('Schrier', 9),
 ('Davis', 8),
 ('Quigley', 8),
 ('Flores', 8),
 ('Turner', 7),
 ('Cuellar', 7),
 ('Wilson', 7),
 ('Castro', 7),
 ('Jayapal', 7),
 ('Stewart', 7),
 ('Garcia', 7),
 ('Long', 6),
 ('Aguilar', 6),
 ('Peters', 6),
 ('Carson', 6),
 ('Clay', 6),
 ('Brown', 6),
 ('McHenry', 6),
 ('King', 6),
 ('Pallone', 6),
 ('Rogers', 5),
 ('Allen', 5),
 ('Lofgren', 5),
 ('Neal', 5),
 ('Meadows', 5),
 ('Demings', 5),
 ('Lewis', 5),
 ('Joyce', 5),
 ('Keating', 5),
 ('Duncan', 5),
 ('Cooper', 5),
 ('Sarbanes', 5),
 ('Schweikert', 5),
 ('Dean', 5),
 ('Upton', 5),
 ('Smucker', 5),
 ('Larsen', 5),
 ('Murphy', 4),
 ('Nunes', 4),
 ('Bishop', 4),
 ('Ocasi

In [34]:
from collections import Counter